In [84]:
import tensorflow as tf
import numpy as np
import random

# Create 3-input XOR training dataset..
training_length = 6
x_train = np.random.randint(0,2, size=(training_length,3),dtype=np.uint8)
y_train=np.empty(training_length).astype(int)
#x_train=np.array([[0,0,0],[0,0,1],[0,1,0],[1,1,0],[1,0,0],[1,0,1],[0,1,1],[1,1,1]])
for i in range(training_length):
  y_train[i]=(x_train[i][0]^x_train[i][1]^x_train[i][2])

# for i in range(training_length):
#     print(y_train[i], x_train[i][0], x_train[i][1], x_train[i][2])

# Create 3-input XOR testing dataset...
testing_length=20
x_test = np.random.randint(0,2, size=(testing_length,3),dtype=np.uint8)
y_test=np.empty(testing_length).astype(int)
for i in range(testing_length):
   y_test[i]=x_test[i][0]^x_test[i][1]^x_test[i][2]
for i in range(testing_length):
  y_test[i]=x_test[i][0]^x_test[i][1]^x_test[i][2]

for i in range(testing_length):
    print(y_test[i], x_test[i][0], x_test[i][1], x_test[i][2])

0 0 1 1
1 0 0 1
1 1 0 0
0 0 0 0
0 0 0 0
0 1 0 1
0 1 0 1
0 0 1 1
0 0 1 1
1 1 1 1
0 0 0 0
0 0 0 0
1 1 1 1
0 1 0 1
1 0 1 0
1 1 0 0
1 0 0 1
0 1 0 1
1 1 0 0
1 1 0 0


In [85]:
def featuresl(n):
      c=[]
      for i in range(n):
        for j in range(2):
          if j==0:
            c.append(f'x{i+1}')
          else:
            c.append(f'~x{i+1}')
      return c 
c=featuresl(3)

In [86]:
import numpy as np
import random
import matplotlib.pyplot as plt

class tsetlinmachine:
    def __init__(self,clauses,features,states,s,T):
        # tsetlin machine hyperparameters.
        self.clauses=clauses
        self.features=features
        self.states=states
        self.s=s
        self.T=T

        # Initialize ta_state for each literal with minimum memory state.
        self.ta_state=np.random.choice([states-1,states],size=(clauses,features,2))
        
        # Initialize clause things.
        self.clauses_output=np.zeros(clauses)
        self.clause_sign=np.zeros(clauses)
        self.feedback=np.zeros(clauses)

        # Assign polarity to clauses. +ve to even and -ve to odd. 
        for j in range(len(self.clause_sign)):
            if j%2==0:
                self.clause_sign[j]=+1
            else:
                self.clause_sign[j]=-1

    # If literal lies in memory state assign 1 or else 0.
    def action(self,state):
        if state<=(self.states-1):
            return 0
        else:
            return 1
            
        # Returns clauses_output vector.
    def clause_output(self):
        return self.clauses_output
        
        # To get current TA state of a literal.
    def get_state(self,clause,feature,l):
        return self.ta_state[clause,feature,l]

        # Calculates all the clauses output for a given literal vector X.
    def get_clause_output(self,X):
        for j in range(self.clauses):
            self.clauses_output[j]=1
            for k in range(self.features):
                action_l=self.action(self.ta_state[j,k,0])
                action_nl=self.action(self.ta_state[j,k,1])
                if(action_l==1 and X[k]==0) or (action_nl==1 and X[k]==1):
                    self.clauses_output[j]=0
                    break
        # Calculates class sum.
    def get_class_sum(self):
        sum=0
        for j in range(self.clauses):
            sum+=(self.clauses_output[j])*(self.clause_sign[j])
        if sum > self.T:
            sum=self.T
        elif sum < -self.T:
            sum=-self.T
        return sum
        
        # Predicts class for given X.
    def get_predicted_class(self,X):
        self.get_clause_output(X)
        if self.get_class_sum()>0:
            return 1
        else:
            return 0
# Training of Tsetlin machine

    # Initialize feedback
    def training(self,X,Y):
        self.get_clause_output(X)
        for j in range(self.clauses):
            self.feedback[j]=0
        if Y==1:
            for j in range(self.clauses):
                if random.random()>1.0*(self.T-self.get_class_sum())/(2*self.T):
                    continue
                elif j%2==0:
                    self.feedback[j]=+1
                else:
                    self.feedback[j]=-1
        else:
            for j in range(self.clauses):
                if random.random()>1.0*(self.T+self.get_class_sum())/(2*self.T):
                    continue
                elif j%2==0:
                    self.feedback[j]=-1
                else:
                    self.feedback[j]=+1

        # Giving feedback to clauses   
        for j in range(self.clauses):
            if self.feedback[j]>0:
                #Type 1 feedback
                if self.clauses_output[j]==1:
                    #Recognize feedback
                    for k in range(self.features):
                       if X[k]==1:
                          # if random.random()<=(self.s-1)/(self.s):
                           if self.get_state(j,k,0)<self.states*2:
                                   self.ta_state[j,k,0]+=1  # memorize literal kth x.
                                   
                           if random.random()<=(1/self.s):
                               if self.get_state(j,k,1)>0:
                                   self.ta_state[j,k,1]-=1  # forget literal kth x'.
                                   
                       elif X[k]==0:
                           #if random.random()<=(self.s-1)/(self.s):
                           if self.get_state(j,k,1)<self.states*2:
                                   self.ta_state[j,k,1]+=1  # memorize literal kth x'.
                                   
                           if random.random()<=(1/self.s):
                               if self.get_state(j,k,0)>0:
                                   self.ta_state[j,k,0]-=1  # forget literal kth x.
                                   
                elif self.clauses_output[j]==0:
                    #Erase feedback
                    for k in range(self.features):
                        if random.random()<=(1/self.s):
                            if self.get_state(j,k,0)>0:
                                self.ta_state[j,k,0]-=1
                             
                        if random.random()<=(1/self.s):
                            if self.get_state(j,k,1)>0:
                                self.ta_state[j,k,1]-=1
                              
         #Reject feedback
        if self.feedback[j]<0:
            if self.clauses_output[j]==1:
                for k in range(self.features):
                    action_l=self.action(self.get_state(j,k,0))
                    action_nl=self.action(self.get_state(j,k,1))
                    if action_l==0 and self.get_state(j,k,0)<self.states*2:
                        self.ta_state[j,k,0]+=1                        
                    if action_nl==0 and self.get_state(j,k,1)<self.states*2 :
                        self.ta_state[j,k,1]+=1  # Try to include literals from excluded state to make clause output zero

In [91]:
clauses = 11
features = x_train.shape[1]
states = 5
s = 13
T = 5
epochs = 700
print("It's training, yeah!")
MNIST_tm = tsetlinmachine(clauses,features,states,s,T) # Create object of class tsetlinmachine
for epoch in range(epochs):

    for i in range(x_train.shape[0]):
        MNIST_tm.training(x_train[i],y_train[i])

It's training, yeah!


In [92]:
# testing_length=20
# x_test = np.random.randint(0,2, size=(testing_length,3),dtype=np.uint8)
# y_test=np.empty(testing_length).astype(int)

 
x_test=np.array([[0,1,1],[1,1,1]])

for i in range(2):
   y_test[i]=(x_test[i][0]^x_test[i][1]^x_test[i][2])
correct_prediction = 0 

total_predictions = x_test.shape[0]

for i in range(x_test.shape[0]):
    prediction = MNIST_tm.get_predicted_class(x_test[i])
    if prediction == y_test[i]:
        correct_prediction +=1

accuracy = correct_prediction / total_predictions
print(f"Test Acccuracy: {accuracy * 100:.2f}%")

Test Acccuracy: 100.00%


In [93]:
def featuresl(n):
      c=[]
      for i in range(n):
        for j in range(2):
          if j==0:
            c.append(f'x{i+1}')
          else:
            c.append(f'~x{i+1}')
      return c 
c=featuresl(3)    
for j in range(clauses):
    for k in range(features):
        for l in range(2):
            if MNIST_tm.ta_state[j,k,l]>=states:
                C.append(c[2*k+l]) 
    print(f'clause {j} :',C)        
    C=[]

clause 0 : ['x1', 'x2', 'x3']
clause 1 : ['~x1', '~x2', '~x3']
clause 2 : ['~x1', 'x2', '~x3']
clause 3 : ['~x1', 'x2', 'x3']
clause 4 : ['~x1', 'x2', '~x3']
clause 5 : ['~x1', '~x2', '~x3']
clause 6 : ['~x1', '~x2', 'x3']
clause 7 : ['~x1', 'x2', 'x3']
clause 8 : ['~x1', '~x2', 'x3']
clause 9 : ['~x1', 'x2', 'x3']
clause 10 : ['x1', 'x2', 'x3']


In [90]:
# import tensorflow as tf
# import numpy as np
# #!pip install matplotlib
# # Binary Classifier

# (x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data() # Load MNIST dataset from keras

# # Extract only images of integers 0 and 1, that is class-0 and class-1. 
# # x --> handwritten digits images, y--> respective labels.

# class_0_train = x_train[y_train == 0]  #class-0 for training. shape(5923,28,28) each image is a matrix of 28 X 28
# class_1_train = x_train[y_train == 1]  #class-1 for training. shape(6742,28,28) each image is a matrix of 28 X 28

# class_0_test = x_test[y_test == 0]     #class-0 for testing. shape(980,28,28)
# class_1_test = x_test[y_test == 1]     #class-1 for testing. shape(1135,28,28)

# # Binarization function to binarize image w.r.t threshold.

# def bin(image,threshold = 128):
#     return np.where(image > 128,1,0)

# class_0_train = bin(class_0_train)   #Binarize class-0 images
# class_1_train = bin(class_1_train)   #Binarize class-1 images

# class_0_train = class_0_train.reshape(class_0_train.shape[0],-1)  #Reshape 2D images to 1D array. shape(5923,784), arrays of features.
# class_1_train = class_1_train.reshape(class_1_train.shape[0],-1)  #Reshape 2D images to 1D array. shape(6742,784), arrays of features.

# # class_0_neg = 1-class_0_train  #Negated features.
# # class_1_neg = 1-class_1_train  #Negated features.

# # class_0_literals = np.concatenate((class_0_train,class_0_neg),axis = 1) #This is a full literal vector. shape(5923,1568)---784+784=1568.
# # class_1_literals = np.concatenate((class_1_train,class_1_neg),axis = 1) #This is a full literal vector. shape(6742,1568)

# combined_train=np.concatenate((class_0_train,class_1_train),axis = 0)  #shape(12665,1584)---5923+6742=12665

# class_0_label=np.zeros(class_0_train.shape[0]) #Creating label for class-0
# class_1_label=np.ones(class_1_train.shape[0])  #Creating label for class-1
# combined_label=np.concatenate((class_0_label,class_1_label),axis=0) #Combine both labels. shape(12665)

# train_indices=np.arange(combined_train.shape[0])  #Creates indices of [0,1,2,....12664]
# np.random.shuffle(train_indices)                  #Shuffles randomly [23,45,76,....1290]
# combined_train=combined_train[train_indices]      #Shuffle data.
# combined_label=combined_label[train_indices]      #Shuffle labels.
# class_0_neg = 1-class_0_train  #Negated features.
# # class_1_neg = 1-class_1_train  #Negated features.

# # class_0_literals = np.concatenate((class_0_train,class_0_neg),axis = 1) #This is a full literal vector. shape(5923,1568)---784+784=1568.
# # class_1_literals = np.concatenate((class_1_train,class_1_neg),axis = 1)
# x_train = combined_train  #Traing literals for training.
# y_train = combined_label  #Labels for training.

# # Similarly create Testing dataset..

# class_0_test=bin(class_0_test)
# class_1_test=bin(class_1_test)

# class_0_test=class_0_test.reshape(class_0_test.shape[0],-1)
# class_1_test=class_1_test.reshape(class_1_test.shape[0],-1)

# # class_0_neg=1-class_0_test
# # class_1_neg=1-class_1_test

# # combined_class_0=np.concatenate((class_0_test,class_0_neg),axis=1)
# # combined_class_1=np.concatenate((class_1_test,class_1_neg),axis=1)

# combined_test=np.concatenate((class_0_test,class_1_test),axis=0)

# class_0_label=np.zeros(class_0_test.shape[0])
# class_1_label=np.ones(class_1_test.shape[0])

# combined_label=np.concatenate((class_0_label,class_1_label),axis=0)
# test_indices=np.arange(combined_label.shape[0])
# test_indices
# np.random.shuffle(test_indices)

# combined_test=combined_test[test_indices]
# combined_label=combined_label[test_indices]

# x_test=combined_test # Shape(2115,1568)
# y_test=combined_label #Shape(2115)

# # Training and Testing datasets are ready to see the Machine ..... 
